# SentencePiece 토크나이저 + KorQuAD 데이터셋 (한국어)

## 1. 개요

이 문서의 목적:
* SentencePiece 토크나이저가 제공하는 여러 가지 옵션을 변경해 가면서 결과가 어떻게 달라지는지 파악합니다.

모델:
* [SentencePiece](https://github.com/google/sentencepiece): Unsupervised text tokenizer for Neural Network-based text generation. By Google.

데이터셋:
* [KorQuAD v1.0](https://korquad.github.io/category/1.0_KOR.html): The Korean Question Answering Dataset. By LG CNS AI Research Center

## 2. 토크나이저 훈련용 데이터 파일 만들기

데이터셋을 다운로드하여 메모리에 적재하고 이로부터 토크나이저 훈련용 데이터 파일을 만듭니다.

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

dataset_name = "KorQuAD/squad_kor_v1"
dataset = load_dataset(dataset_name, trust_remote_code=True)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})


In [3]:
samples = dataset["train"].shuffle(seed=42).select(range(3))

for row in samples:
    print(f"\n'>>> context: {row['context']}'")
    print(f"'>>> question: {row['question']}'")
    print(f"'>>> answers: {row['answers']}'")


'>>> context: 9월 26일 환경부를 비롯한 12개 관계부처가 합동으로 '미세먼지 관리 종합대책'을 확정하고 발전·산업·수송·생활 등 4개 부분에서 저감 대책을 실시하는 관련 로드맵을 발표했다. 7조 2000억 원의 예산을 투입해 미세먼지 국내 배출량을 30% 감축하고 미세먼지 '나쁨' 일수를 70%까지 줄이기로 한 것인데 이를 위해 공정률 10% 미만인 석탄발전소 9기 중 4기를 LNG 등 친환경 연료로 전환하고 남은 5기도 최고 수준의 배출 기준을 적용하며 30년이 넘은 노후 석탄발전소 7기는 임기 내 폐쇄하기로 했다. 또한 대기배출총량제를 전국으로 확대·강화하고 먼지총량제를 새로 도입하며, 노후 경유차 221만 대를 임기 내 77% 조기 폐차하고 친환경 차를 2022년까지 200만 대 보급하며 미세먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행하기로 했다. 국제적으로는 미세먼지를 한중 양국의 정상의제로 격상하고 동북아 지역에서 협약체결을 추진하면서 미세먼지 환경기준도 선진국 수준으로 강화할 것도 포함했다.'
'>>> question: 미세먼지 해결을 위해 전국으로 확대 강화된 기존의 제도는?'
'>>> answers: {'text': ['대기배출총량제'], 'answer_start': [290]}'

'>>> context: 프리스틀리는 워링턴 거주 시절에는 다른 일 때문에 신학 연구에 몰두하지 못하였으나, 리즈에 오면서 그는 신학 연구에 많은 시간을 투자하였고, 결과적으로 그의 신앙은 아리우스주의에서 유니테리언으로 정립되었다. 리즈에서 프리스틀리는 삼위일체와 예수의 선재성(先在性, 성자인 예수는 천지창조전부터 성부와 같이 존재했다는 교리)등을 완전히 부정하였고, 기독교 교리와 성서를 새롭게 해석하기 시작했다. 그는 오래전부터 써오던 신학 교육에 대한 책인 《자연과 계시 종교의 원리》(Institutes of Natural and Revealed Religion)를 출판하기 시작하였는데, 1772년에 1권이 출판되었고 마지막 3권은 177

훈련용 데이터 파일을 네 개의 파일로 분리해서 만듭니다. 물론 하나의 파일로 만들어서 사용해도 됩니다.

In [4]:
with open('korquad_train_context.txt', 'w', encoding='utf8') as f:
    f.write("\n".join(dataset["train"]["context"]))
with open('korquad_train_question.txt', 'w', encoding='utf8') as f:
    f.write("\n".join(dataset["train"]["question"]))

with open('korquad_validation_context.txt', 'w', encoding='utf8') as f:
    f.write("\n".join(dataset["validation"]["context"]))
with open('korquad_validation_question.txt', 'w', encoding='utf8') as f:
    f.write("\n".join(dataset["validation"]["question"]))

## 3. SentencePiece 훈련

단어 사전 크기와 알고리즘을 변경해 가면서 토크나이저 훈련을 수행하고 결과를 저장합니다.

In [5]:
!pip install sentencepiece

In [6]:
import os
import sentencepiece as spm

filenames = [
    "korquad_train_context.txt",
    "korquad_train_question.txt",
    "korquad_validation_context.txt",
    "korquad_validation_question.txt"
]
input_files = ",".join(filenames)

output_dir = "sentence_piece_korquad"
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

vocab_sizes = [5000, 10000, 20000, 30000, 40000, 50000]
algorithms = ["bpe", "unigram"]
for vocab_size in vocab_sizes:
    for algorithm in algorithms:
        model_prefix = os.path.join(output_dir, f"korquad_{vocab_size}_{algorithm}")
        spm.SentencePieceTrainer.Train(f"--input={input_files} \
            --model_prefix={model_prefix} \
            --vocab_size={vocab_size} \
            --model_type={algorithm} \
            --max_sentence_length=24140 \
            --minloglevel=1")
        print(f"Saved tokenizer: {model_prefix}")

Saved tokenizer: sentence_piece_korquad/korquad_5000_bpe
Saved tokenizer: sentence_piece_korquad/korquad_5000_unigram
Saved tokenizer: sentence_piece_korquad/korquad_10000_bpe
Saved tokenizer: sentence_piece_korquad/korquad_10000_unigram
Saved tokenizer: sentence_piece_korquad/korquad_20000_bpe
Saved tokenizer: sentence_piece_korquad/korquad_20000_unigram
Saved tokenizer: sentence_piece_korquad/korquad_30000_bpe
Saved tokenizer: sentence_piece_korquad/korquad_30000_unigram
Saved tokenizer: sentence_piece_korquad/korquad_40000_bpe
Saved tokenizer: sentence_piece_korquad/korquad_40000_unigram
Saved tokenizer: sentence_piece_korquad/korquad_50000_bpe
Saved tokenizer: sentence_piece_korquad/korquad_50000_unigram


단어 사전의 내용을 확인해 봅니다.

In [7]:
import pandas as pd
import csv

vocab_file = "sentence_piece_korquad/korquad_10000_bpe.vocab"
vocab_list = pd.read_csv(vocab_file, sep='\t', header=None, quoting=csv.QUOTE_NONE)
print(len(vocab_list))
print(vocab_list.sample(5))

10000
        0     1
1295  ▁이끌 -1292
3411  민주당 -3408
1431   ▁펼 -1428
3122  ▁혼란 -3119
997    ▁봉  -994


## 4. 훈련 결과 이용

In [8]:
sp = spm.SentencePieceProcessor()

texts = [
    "미세먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행",
    "가뜩이나 어려운 조건 속에서"
]

for text in texts:
    print("#" * 80)
    print("TEXT: " + text)
    for vocab_size in vocab_sizes:
        for algorithm in algorithms:
            filename = f"korquad_{vocab_size}_{algorithm}.model"
            model_file = os.path.join(output_dir, filename)
            sp.load(model_file)
            print(f"{vocab_size:5} {algorithm:7}: {sp.encode_as_pieces(text)}")
            #print(f"    {sp.encode_as_ids(text)}")

################################################################################
TEXT: 미세먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행
 5000 bpe    : ['▁미', '세', '먼', '지가', '▁심', '하면', '▁차', '량', '▁2', '부', '제', '와', '▁같은', '▁비', '상', '저', '감', '조', '치를', '▁시', '행']
 5000 unigram: ['▁미', '세', '먼', '지', '가', '▁심', '하면', '▁차량', '▁2', '부', '제', '와', '▁같은', '▁비', '상', '저', '감', '조', '치', '를', '▁시행']
10000 bpe    : ['▁미', '세', '먼', '지가', '▁심', '하면', '▁차량', '▁2', '부', '제와', '▁같은', '▁비상', '저', '감', '조', '치를', '▁시행']
10000 unigram: ['▁미', '세', '먼', '지', '가', '▁심', '하면', '▁차량', '▁2', '부', '제', '와', '▁같은', '▁비상', '저', '감', '조', '치', '를', '▁시행']
20000 bpe    : ['▁미세먼', '지가', '▁심', '하면', '▁차량', '▁2', '부', '제와', '▁같은', '▁비상', '저', '감', '조치를', '▁시행']
20000 unigram: ['▁미세먼지', '가', '▁심', '하면', '▁차량', '▁2', '부', '제', '와', '▁같은', '▁비상', '저', '감', '조치', '를', '▁시행']
30000 bpe    : ['▁미세먼', '지가', '▁심', '하면', '▁차량', '▁2', '부', '제와', '▁같은', '▁비상', '저', '감', '조치를', '▁시행']
30000 unigram: ['▁미세먼지', '가', '▁심', '하면', '▁차량', '▁2'